In [ ]:
!pip install Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.9 MB/s eta 0:00:00


In [3]:
import sys
from Bio import Entrez
import pandas as pd
import os
import scanpy as sc
import numpy as np

In [9]:
adata_est = sc.read("datasets/masked.h5ad")

# **Create input files for R (Estimate)**

In [10]:
# !mkdir R_time

# Transpose the expression matrix
expression_matrix = adata_est.X.T

# Get the gene and cell names
gene_names = adata_est.var_names
cell_names = adata_est.obs_names
lines = []

# Add the header line with cell names
header = "gene\t" + "\t".join(cell_names)
lines.append(header)

# Add the data lines with gene names and expression values
for i in range(expression_matrix.shape[0]):
    gene_name = gene_names[i]
    expression_values = "\t".join(expression_matrix[i].astype(str))
    line = gene_name + "\t" + expression_values
    lines.append(line)

# Write the lines to a text file in the Seurat_files directory
output_directory = "R_time"
os.makedirs(output_directory, exist_ok=True)
output_file = os.path.join(output_directory, "input_Estimate_format.txt")
with open(output_file, "w") as file:
    file.write("\n".join(lines))

# **Create input files for R (Genefu)**

In [4]:
adata_gen = sc.read("datasets/tumour_cellcounts.h5ad")

In [5]:
tumor_indices = np.where(adata_gen.obs['type'] == 'Tumor')[0]
expr_data = adata_gen.X[tumor_indices, :]

In [6]:
# Save gene expression data to a CSV file
np.savetxt('R_time/gene_expression.csv', expr_data, delimiter=',')

# Save features.csv
features = pd.DataFrame({'GeneID': adata_gen.var_names})
features.to_csv('R_time/gene_names.csv', sep=',', index=False)

# Convert cell_names to a one-dimensional string array
cell_names_arr = np.array(adata_gen.obs_names)[tumor_indices].astype(str)

barcodes = pd.DataFrame({'CellID': cell_names_arr})
barcodes.to_csv('R_time/cell_names.csv', sep=',', index=False)

# **API for Entrez ID**

In [7]:
Entrez.email = "s.alvanakis@gmail.com"

In [8]:
dfs = pd.read_csv('R_time/gene_names.csv')

In [9]:
genes_list = dfs["GeneID"].values.tolist()

In [10]:
def get_entrez_gene_id(gene_name):
    handle = Entrez.esearch(db="gene", term=gene_name)
    record = Entrez.read(handle)
    handle.close()
    if len(record["IdList"]) > 0:
        return record["IdList"][0]
    else:
        return None


In [11]:
entrez_gene_ids = []

In [ ]:
for gene_name in genes_list:
    entrez_gene_id = get_entrez_gene_id(gene_name)
    entrez_gene_ids.append(entrez_gene_id)


In [ ]:
dfs['Entrez_ID'] = entrez_gene_ids

In [46]:
# Save DataFrame as a CSV file
dfs.to_csv('R_time/Entrez_id.csv', sep=',', index=False)